In [17]:
#
# Chemical Reaction Optimization: Lennard-Jones potential
# Main program
# author: Luis Rincon (USFQ)
# date: 22 March 2022
#
# Import libraries
#
import numpy as np                           # python ecosystem for scientific computing
from scipy.optimize import minimize          # scipy unconstrained minimization

In [28]:
# Parameter of the simulation
n = 20                                       # number of atoms in the cluster
temp = 4.0                                   # initial temperature
PopSize = 100                                # size of the initial population
ke_loss_rate = 0.75                          # percentage of the upper limit of reduction of ke
molecoll = 0.50                              # probability that the reaction is uni-molecular or inter-molecular
buffer = 0.0                                 # energy of surrounding
n_cro = 1000                                 # number of CRO steps
min_hit = 2                                  # number of collisions before locating a better local minima
# Arrays
x_pop = np.zeros((PopSize,3*n))              # coordinates of the population
pe_pop = np.zeros(PopSize)                   # potential energy of the population
ke_pop = np.zeros(PopSize)                   # kinetic energy of the population
NumHit = np.zeros(PopSize,dtype='int')       # Number of innefectives collisions
min_struct = np.array(3*n)                   # best solution

In [29]:
def lj(iatom,jatom,x):
    """Lennard-Jones Potential"""
    xr = x[3*iatom] - x[3*jatom]
    yr = x[3*iatom+1] - x[3*jatom+1]
    zr = x[3*iatom+2] - x[3*jatom+2]
    r2 = np.power(xr,2) + np.power(yr,2) + np.power(zr,2)
    if r2<=0.010:
        r2 = 0.010
    r2i = 1.0/r2
    r6i = np.power(r2i,3)
    return 4.0*r6i*(r6i-1.0)

In [30]:
def atom_lj(natoms,iatom,x):
    """Atomic energies"""
    enei = 0.0
    for jatom in range(natoms):
        if jatom!=iatom:
            enei = enei + 0.5*lj(iatom,jatom,x)
    return enei 

In [31]:
def ene_lj(x):                                              
    """Total energy"""
    natoms = int(len(x)/3)                                          # number of atoms   
    ene = 0.0
    for iatom in range(natoms):
        ene = ene + atom_lj(natoms,iatom,x)                        # update energy
    return ene        

In [32]:
def dene_lj(x):
    """The Lennard-Jones gradient vector"""
    natoms = int(len(x)/3)                                  
    dene = np.zeros_like(x)
    for iatom in range(natoms-1):
        for jatom in range(iatom+1,natoms):
            xr = x[3*iatom] - x[3*jatom]
            yr = x[3*iatom+1] - x[3*jatom+1]
            zr = x[3*iatom+2] - x[3*jatom+2]
            r2 = np.power(xr,2) + np.power(yr,2) + np.power(zr,2)
            if r2==0.0:
                r2=0.10
            r2i = 1.0/r2
            r6i = np.power(r2i,3)
            df = -48.0*r2i*r6i*(r6i-0.5)
            dene[3*iatom] = dene[3*iatom] + df*xr
            dene[3*iatom+1] = dene[3*iatom+1] + df*yr
            dene[3*iatom+2] = dene[3*iatom+2] + df*zr
            dene[3*jatom] = dene[3*jatom] - df*xr
            dene[3*jatom+1] = dene[3*jatom+1] - df*yr
            dene[3*jatom+2] = dene[3*jatom+2] - df*zr
    return dene

In [33]:
def cm(x):
    """center of mass"""
    natoms = int(len(x)/3)
    xcm = 0.0
    ycm = 0.0
    zcm = 0.0
    for i in range(natoms):
        xcm = xcm + x[3*i]
        ycm = ycm + x[3*i+1]
        zcm = zcm + x[3*i+2]
    xcm = xcm/natoms
    ycm = ycm/natoms
    zcm = zcm/natoms
    return xcm,ycm,zcm

In [34]:
def dmc_search(x):
    natoms = int(len(x)/3)
    eatom = np.zeros(natoms)
    for iatom in range(natoms):
        eatom[iatom] = atom_lj(natoms,iatom,x)
    emax = np.amax(eatom)
    emin = np.amin(eatom)
    patom = np.zeros(natoms)
    for iatom in range(natoms):
        patom[iatom] = np.exp(-4.0*(emax-eatom[iatom])/(emax-emin))
    sum_patom = 0.0
    for iatom in range(natoms):
        sum_patom = sum_patom + patom[iatom]
    for iatom in range(natoms):
        patom[iatom] = patom[iatom]/sum_patom
    o = int(np.random.choice(natoms,size=1,p=patom))
    xn = np.zeros(3*natoms)
    xn = x
    xmax = int(np.amax(xn)) + 1
    xmin = int(np.amin(xn)) - 1
    min_eneo = 0.0
    min_ix = xmin
    min_iy = xmin
    min_iz = xmin
    for ix in range(xmax-xmin+1):
        xn[3*o] = float(xmin + ix)
        for iy in range(xmax-xmin+1):
            xn[3*o+1] = float(xmin + iy)
            for iz in range(xmax-xmin+1):
                xn[3*o+2] = float(xmin + iz)
                eneo = atom_lj(natoms,o,xn)
                if eneo<min_eneo:
                    min_eneo = eneo
                    min_ix = ix
                    min_iy = iy
                    min_iz = iz
    xn[3*o] = float(xmin + min_ix)
    xn[3*o+1] = float(xmin + min_iy)
    xn[3*o+2] = float(xmin + min_iz)
    return xn

In [35]:
# Initialization                         
rc = 0.60*np.power(float(n),1.0/3.0)
min_pe = 0.0
for i in range(PopSize):
    x_pop[i,0] = rc*(2.0*np.random.rand()-1.0)
    x_pop[i,1] = rc*(2.0*np.random.rand()-1.0)
    x_pop[i,2] = rc*(2.0*np.random.rand()-1.0)
    for j in range(1,n):
        check = True
        while check:
            check = False
            x_pop[i,3*j] = rc*(2.0*np.random.rand()-1.0)
            x_pop[i,3*j+1] = rc*(2.0*np.random.rand()-1.0)
            x_pop[i,3*j+2] = rc*(2.0*np.random.rand()-1.0)
            for k in range(j):
                dx = x_pop[i,3*j]-x_pop[i,3*k]
                dy = x_pop[i,3*j+1]-x_pop[i,3*k+1]
                dz = x_pop[i,3*j+2]-x_pop[i,3*k+2]
                rij = np.sqrt(np.power(dx,2)+np.power(dy,2)+np.power(dz,2))
                if rij<=1.0:
                    check=True
    opt = minimize(ene_lj,x_pop[i,:], method='L-BFGS-B', jac=dene_lj, options={'disp': True})
    x_pop[i,:] = opt.x
    pe_pop[i] = opt.fun
    ke_pop[i] = 4.0*np.power(np.random.normal(loc=2.0*temp,scale=1.0),2)
    print(i,pe_pop[i],ke_pop[i])
    xcm,ycm,zcm = cm(x_pop[i,:])
    for j in range(n):
        x_pop[i,3*j] = x_pop[i,3*j] - xcm
        x_pop[i,3*j+1] = x_pop[i,3*j+1] - ycm
        x_pop[i,3*j+2] = x_pop[i,3*j+2] - zcm
    if pe_pop[i] < min_pe:
        min_pe = pe_pop[i]
        min_struct = x_pop[i,:]
print(min_pe)

0 -74.7575634506192 286.1777837567554
1 -72.11477805513394 295.94691130950264
2 -75.638404830502 319.65057534354247
3 -71.36354292854355 255.4737776834702
4 -72.90567881705601 287.3603288496108
5 -70.39730400858684 269.04366148074934
6 -74.12953340443639 399.38459485430684
7 -72.29830647630753 221.91539305388082
8 -73.87001661616459 280.83252983433573
9 -73.56326527533331 302.3929360913502
10 -71.35611078348309 216.04271566926874
11 -75.25153757927598 206.07374240685928
12 -76.29990060267036 296.35942976533676
13 -72.70721580649116 287.7014031087374
14 -73.42193576833208 193.32488267707757
15 -74.73496397366402 320.5043344083908
16 -72.9166782737426 318.2067667761352
17 -72.73577617277697 343.89043384650586
18 -73.26050106270638 349.9916579373749
19 -76.29990064579931 264.51180677478084
20 -75.36500315739926 295.03118142138754
21 -73.40166011512494 190.11009748391592
22 -74.00085604742327 351.9081277958584
23 -71.64316415051468 186.53107133954586
24 -71.72451156405145 201.0456481858326

In [36]:
# CRO main loop
for i_cro in range(n_cro):
    xmin = np.array(3*n)
    t = np.random.rand()
    if t<molecoll:                                     # unimolecular collision
        mx = np.random.randint(PopSize-1)              # select a molecule as random
        xmin = x_pop[mx]
        emin = pe_pop[mx]
        if NumHit[mx]>min_hit:                         # Decomposition criterion
            # Trigger Decomposition
            xn = np.array(3*n)
            xn = x_pop[mx]
            opt = minimize(ene_lj,xn, method='L-BFGS-B', jac=dene_lj, options={'disp': True})
            xn = opt.x
            xcm,ycm,zcm = cm(xn)
            for icm in range(n):
                xn[3*icm] = xn[3*icm] - xcm
                xn[3*icm+1] = xn[3*icm+1] - ycm
                xn[3*icm+2] = xn[3*icm+2] - zcm
            pen = opt.fun
            e_dec = ke_pop[mx]-pen
            delta = np.random.rand()
            buffer = buffer + delta*e_dec
            ken = (1.0-delta)*e_dec
            # insert a new element
            pval = np.random.randint(PopSize-1)
            x_pop = np.insert(x_pop,pval,[xn],axis=0)
            pe_pop = np.insert(pe_pop,pval,pen)
            ke_pop = np.insert(ke_pop,pval,ken)
            NumHit = np.insert(NumHit,pval,0)
            PopSize = PopSize + 1
            xmin = x_pop
            emin = pen
        else:                  
            # Trigger OnwallIneffectiveCollision
            xn = np.array(3*n)
            xn = dmc_search(x_pop[mx])
            pen = ene_lj(xn)
            den = pe_pop[mx]+ke_pop[mx]
            if den>pen:
                a = ke_loss_rate + (1.0-ke_loss_rate)*np.random.rand()
                ken = a*(den-pen)
                buffer = buffer + (1.0-a)*(den-pen)
                x_pop[mx] = xn
                pe_pop[mx] = pen
                ke_pop[mx] = ken
                NumHit[mx] = NumHit[mx] + 1
                xmin = xn
                emin = pen
    else:                                              # bimolecular collision
        mx1 = np.random.randint(PopSize-1)             # select a molecule as random
        mx2 = np.random.randint(PopSize-1)             # select a molecule as random
        if (NumHit[mx1]>=min_hit) and (NumHit[mx2]>=min_hit):  # Synthesis criterion
            # Trigger Synthesis
            xn1 = np.array(3*n)
            xn1 = x_pop[mx1]
            opt = minimize(ene_lj,xn1, method='L-BFGS-B', jac=dene_lj, options={'disp': True})
            xn1 = opt.x
            xcm,ycm,zcm = cm(xn1)
            for icm in range(n):
                xn1[3*icm] = xn1[3*icm] - xcm
                xn1[3*icm+1] = xn1[3*icm+1] - ycm
                xn1[3*icm+2] = xn1[3*icm+2] - zcm
            pen1 = opt.fun
            xn2 = np.array(3*n)
            xn2 = x_pop[mx2]
            opt = minimize(ene_lj,xn2, method='L-BFGS-B', jac=dene_lj, options={'disp': True})
            xn2 = opt.x
            xcm,ycm,zcm = cm(xn2)
            for icm in range(n):
                xn2[3*icm] = xn2[3*icm] - xcm
                xn2[3*icm+1] = xn2[3*icm+1] - ycm
                xn2[3*icm+2] = xn2[3*icm+2] - zcm
            pen2 = opt.fun
            xn = np.array(3*n)
            if pen1<pen2:
                xn = xn1
                pen = pen1
            else:
                xn = xn2
                pen = pen2
            e_dec = pe_pop[mx1]+ke_pop[mx1]+pe_pop[mx2]+ke_pop[mx2]-pen
            if e_dec>0:
                a = ke_loss_rate + (1.0-ke_loss_rate)*np.random.rand()
                ken = a*e_dec
                buffer = buffer + (1.0-a)*e_dec
                if pen1<pen2:
                    x_pop[mx1] = xn
                    pe_pop[mx1] = pen
                    ke_pop[mx1] = ken
                    NumHit[mx1] = 0
                    x_pop = np.delete(x_pop,mx2,axis=0)
                    pe_pop = np.delete(pe_pop,mx2)
                    ke_pop = np.delete(ke_pop,mx2)
                    NumHit = np.delete(NumHit,mx2)
                    xmin = xn
                    emin = pen
                else:
                    x_pop[mx2] = xn
                    pe_pop[mx2] = pen
                    ke_pop[mx2] = ken
                    NumHit[mx2] = 0
                    x_pop = np.delete(x_pop,mx1,axis=0)
                    pe_pop = np.delete(pe_pop,mx1)
                    ke_pop = np.delete(ke_pop,mx1)
                    NumHit = np.delete(NumHit,mx1)
                    xmin = xn
                    emin = pen
                PopSize = PopSize - 1
        else:
            # Trigger IntermolecularIneffectiveCollision
            xn1 = np.array(3*n)
            xn1 = dmc_search(x_pop[mx1])
            pen1 = ene_lj(xn1)
            xn2 = np.array(3*n)
            xn2 = dmc_search(x_pop[mx2])
            pen2 = ene_lj(xn2)
            if pe_pop[mx1]+ke_pop[mx1]+pe_pop[mx2]+ke_pop[mx2]>pen1+pen2:
                a = ke_loss_rate + (1.0-ke_loss_rate)*np.random.rand()
                ken = a*(pe_pop[mx1]+ke_pop[mx1]+pe_pop[mx2]+ke_pop[mx2]-pen1-pen2)
                buffer = buffer + (1.0-a)*(pe_pop[mx1]+ke_pop[mx1]+pe_pop[mx2]+ke_pop[mx2]-pen1-pen2)
                delta = np.random.rand()
                ken1 = delta*ken
                ken2 = (1.0-delta)*ken
                x_pop[mx1] = xn1
                pe_pop[mx1] = pen1
                ke_pop[mx1] = ken1
                NumHit[mx1] = NumHit[mx1] + 1
                x_pop[mx2] = xn2
                pe_pop[mx2] = pen2
                ke_pop[mx2] = ken2
                NumHit[mx2] = NumHit[mx2] + 1
                if pen2>pen1:
                    xmin = xn1
                    emin = pen1
                else:
                    xmin = xn2
                    emin = pen2
    # Check for any new minimum solution
    if emin<min_pe:
        min_pe = emin 
        min_struct = xmin
    print(i_cro,emin,min_pe,PopSize,buffer)
# end of CRO

0 -68.98173526768474 -77.17704246678744 100 74.38855722882555
1 -70.86080877502569 -77.17704246678744 100 95.2990064555703
2 -68.02211691468092 -77.17704246678744 100 121.50434814005257
3 -72.15466292951973 -77.17704246678744 100 158.4194806818355
4 -71.78183617971904 -77.17704246678744 100 199.96355913819772
5 -73.88539033395082 -77.17704246678744 100 227.96570017075555
6 -73.16766981115597 -77.17704246678744 100 268.96908739856906
7 -72.993365137739 -77.17704246678744 100 302.61922304864686
8 -71.278918364591 -77.17704246678744 100 336.07188159269083
9 -71.12035255353857 -77.17704246678744 100 361.1013861574793
10 -73.34899227769432 -77.17704246678744 100 443.3438210597466
11 -70.64231086096801 -77.17704246678744 100 484.92885641208176
12 -70.05201386856525 -77.17704246678744 100 518.4912081246173
13 -72.38158718877978 -77.17704246678744 100 544.3477938723511
14 -70.61317920351372 -77.17704246678744 100 577.7854551310594
15 -70.83375866713544 -77.17704246678744 100 586.1214494314846


131 -72.25562737140913 -77.17704246678744 98 6434.228754211561
132 -67.5475190790939 -77.17704246678744 98 6562.340261341371
133 -75.60043659239308 -77.17704246678744 99 6864.66260421614
134 -73.56326517842922 -77.17704246678744 100 6871.3453272183815
135 -72.003908039815 -77.17704246678744 100 6885.611964810739
136 -73.09292834569145 -77.17704246678744 99 6921.900472632426
137 -77.17704243540958 -77.17704246678744 100 7039.058796488768
138 -71.48470605161587 -77.17704246678744 101 7131.288096349651
139 -75.60043659239308 -77.17704246678744 102 7141.3209992795
140 -67.49288244596566 -77.17704246678744 103 7212.077441184275
141 -73.69187756323376 -77.17704246678744 103 7274.891525435665
142 -73.56326517842922 -77.17704246678744 102 7304.221494974681
143 -69.73551940197119 -77.17704246678744 102 7313.75582040254
144 -73.7796271864116 -77.17704246678744 103 7580.4837568466655
145 -70.97700553893381 -77.17704246678744 103 7654.184905670242
146 -72.96257794734977 -77.17704246678744 103 7754

259 -72.37785009019424 -77.17704246678744 109 13142.005204131943
260 -70.10237502534324 -77.17704246678744 109 13212.578189457006
261 -73.60155497566417 -77.17704246678744 110 13337.198344780532
262 -71.230065631949 -77.17704246678744 111 13406.92992659012
263 -70.73876093606222 -77.17704246678744 111 13407.218374963162
264 -77.17704253929436 -77.17704253929436 112 13463.411567880497
265 -73.95541738205365 -77.17704253929436 113 13640.840935220886
266 -72.26954496327855 -77.17704253929436 114 13801.518853782207
267 -74.5267670664687 -77.17704253929436 115 14015.371310095441
268 -66.26634605911451 -77.17704253929436 115 14075.507924223168
269 -69.74745124761225 -77.17704253929436 114 14082.75825607509
270 -70.77087678085363 -77.17704253929436 114 14090.425890016053
271 -69.63547747741039 -77.17704253929436 114 14091.933797639731
272 -75.58928905497339 -77.17704253929436 113 14120.86646539759
273 -66.97344784511263 -77.17704253929436 113 14122.48131582767
274 -74.63142171795633 -77.17704

KeyboardInterrupt: 